<a href="https://colab.research.google.com/github/konkolyseismolab/seismolab/blob/master/docs/source/query_gaia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

[Open this tutorial in Google Colab](https://colab.research.google.com/github/konkolyseismolab/seismolab/blob/master/docs/source/query_gaia.ipynb)

# Query Gaia catalog

The *Gaia* module combines Gaia data with the Bailer-Jones distance catalog, galactic extinction maps and magnitudes from the Simbad catalog. Simply passing the Gaia ID of one or more targets, the code automatically queries

- the Gaia parallax,
- the Gaia G, BP and RP brightness,
- the Johnson B, V and 2MASS JHK brightness,

and delivers

- the extinction corrected apparent brightness,
- the extinction corrected absolute brightness,
- extinction values in each band,

with the associated upper and lower errorbars.

__Note:__
Both Gaia DR2 and DR3 catalogs can be used. However, there is a subtle difference between them. In case of DR3, the distance are queried from the [Bailer-Jones catalog](https://ui.adsabs.harvard.edu/abs/2021AJ....161..147B/abstract). If DR2 is used, the distances are estimated from the Gaia parallaxes in a probabilistic way as defined by [Bailer-Jones+ (2018, AJ, 156, 58)](https://ui.adsabs.harvard.edu/abs/2018AJ....156...58B/abstract).

## Dependencies

To estimate the extinction values, the dust maps implemented in the `mwdust` code can be used (https://github.com/jobovy/mwdust). Thus, `mwdust` must be installed prior to `seismolab`.

To install `mwdust`, follow their [instructions](https://github.com/jobovy/mwdust#installation).

## __How to use__

In [1]:
from seismolab.gaia import query_gaia

After importing the relevant module, we can pass a list of __Gaia DR3 IDs__ to perform the query and all the calculations.

In [2]:
targets = [1022096392749670016,3107142829863485952]

result_table = query_gaia(targets)

Using Combined19 map from mwdust
Querying Gaia DR3 and BJ catalogs...
Calculating distances, absolute magnitudes...


  0%|          | 0/2 [00:00<?, ?it/s]

The result is an Astropy Table.

In [3]:
result_table

Source,ra,dec,parallax,sig_parallax,Gmag,sig_Gmag,BPmag,sig_BPmag,RPmag,sig_RPmag,Bmag,sig_Bmag,Vmag,sig_Vmag,Jmag,sig_Jmag,Hmag,sig_Hmag,Kmag,sig_Kmag,dist,distep,distem,lon_deg,lat_deg,aG,aGep,aGem,MG,MGep,MGem,mG,mGep,mGem,aBP,aBPep,aBPem,MBP,MBPep,MBPem,mBP,mBPep,mBPem,aRP,aRPep,aRPem,MRP,MRPep,MRPem,mRP,mRPep,mRPem,aB,aBep,aBem,MB,MBep,MBem,mB,mBep,mBem,aV,aVep,aVem,MV,MVep,MVem,mV,mVep,mVem,aJ,aJep,aJem,MJ,MJep,MJem,mJ,mJep,mJem,aH,aHep,aHem,MH,MHep,MHem,mH,mHep,mHem,aK,aKep,aKem,MK,MKep,MKem,mK,mKep,mKem
,deg,deg,mas,mas,mag,,mag,,mag,,mag,,mag,,mag,,mag,,mag,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
str21,float64,float64,float64,float32,float32,float64,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1022096392749670016,145.93959651092428,55.952388737534136,19.266700572347492,0.01887033,7.7452164,0.007181938969436053,8.088248,0.024051615865364074,7.2756886,0.02388009262163658,8.54,-99.000,7.75,-99.000,6.573,0.029,6.223,0.034,6.184,0.017,51.831443786621094,0.0460662841796875,0.050594329833984375,158.92257837077454,45.90248575416284,0.0,0.0,0.0,4.172249913215637,0.00930267223054404,0.009111341320815036,7.745216369628906,0.007181938969435642,0.007181938969435642,0.0,0.0,0.0,4.515281796455383,0.026172349126472483,0.02598101821674348,8.088248252868652,0.024051615865364084,0.024051615865364084,0.0,0.0,0.0,3.702722191810608,0.026000825882745104,0.0258094949730161,7.275688648223877,0.023880092621636706,0.023880092621636706,0.0,0.0,0.0,4.967033505439758,0.022120733261107972,0.021929402351378968,8.539999961853027,0.019999999999999574,0.019999999999999574,0.0,0.0,0.0,4.177033543586731,0.022120733261107972,0.021929402351378968,7.75,0.019999999999999574,0.019999999999999574,0.0,0.0,0.0,3.0000334978103638,0.03112053871154785,0.030929207801818848,6.572999954223633,0.028999805450439453,0.028999805450439453,0.0,0.0,0.0,2.6500335931777954,0.03612065315246582,0.035929322242736816,6.2230000495910645,0.03399991989135742,0.03399991989135742,0.0,0.0,0.0,2.61103355884552,0.01912093162536621,0.018929600715637207,6.184000015258789,0.017000198364257812,0.017000198364257812
3107142829863485952,101.76822041702867,-1.2075151625364267,0.432227248321794,0.030997224,12.621653,0.0006344411880043353,12.802992,0.0020885144822050884,12.291391,0.0017593474516210648,13.01,0.020,12.72,0.020,11.903,0.023,11.838,0.023,11.8,0.024,2190.859619140625,197.47900390625,151.5408935546875,213.5092718420073,-1.5216790901099373,1.0972506348348021,0.2813463101744822,0.01746006095976571,-0.17867081801656326,0.17374130314264136,0.46938729127948164,11.524401968314612,0.018094502147770086,0.2819807513624877,1.3107321146267983,0.3360851498919084,0.020857096725901858,-0.21081303389254558,0.1785924122029794,0.5255802042911046,11.492259752438631,0.022945611208106342,0.33817366437411245,0.7576583159506094,0.19427135861061418,0.012056279543791626,-0.16933972960112342,0.1694624279902861,0.383437245979227,11.533733056730055,0.013815626995413055,0.1960307060622366,1.5085403078773538,0.3868051982805827,0.024004730459570967,-0.20161286532669465,0.19965198921811478,0.5942121959612479,11.501459921004482,0.04400518822324351,0.4068056560442539,1.1924453025900386,0.305755198782921,0.01897485133607879,-0.17551782189240583,0.19462211009462393,0.5131621964635862,11.527554964438771,0.0389753090

The columns are named according to the followings:

- __Source:__ Gaia DR3 (or DR2) ID 
- __ra, dec:__ equatorial coordinates in deg
- __parallax, sig_parallax:__ parallax and its error in mas
- __\<filter\>mag:__ observed magnitude in given filter
- __sig_\<filter\>mag:__ observed magnitude error in given filter
- __dist:__ distance in pc
- __distep:__ upper distance errorbar in pc
- __distem:__ lower distance errorbar in pc
- __lon_deg, lat_deg:__ galactic coordinates in deg
- __a\<filter>:__ extinction in given filter
- __a\<filter>ep:__ upper extinction errorbar in given filter
- __a\<filter>em:__ lower extinction errorbar in given filter
- __M\<filter>:__ extinction corrected absolute magnitude in given filter
- __M\<filter>ep:__ upper absolute magnitude errorbar in given filter
- __M\<filter>em:__ lower absolute magnitude errorbar in given filter
- __m\<filter>:__ extinction corrected apparent magnitude in given filter
- __m\<filter>ep:__ upper apparent magnitude errorbar in given filter
- __m\<filter>em:__ lower apparent magnitude errorbar in given filter

### Using photodistances

The [Bailer-Jones catalog](https://ui.adsabs.harvard.edu/abs/2021AJ....161..147B/abstract) contains two kinds of distances. By default, the geometric ones are returned, but there is an option to use the __photodistances__ instead.

In [4]:
result_table = query_gaia(targets,use_photodist=True)

Using Combined19 map from mwdust
Querying Gaia DR3 and BJ catalogs...
Calculating distances, absolute magnitudes...


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
result_table

Source,ra,dec,parallax,sig_parallax,Gmag,sig_Gmag,BPmag,sig_BPmag,RPmag,sig_RPmag,Bmag,sig_Bmag,Vmag,sig_Vmag,Jmag,sig_Jmag,Hmag,sig_Hmag,Kmag,sig_Kmag,dist,distep,distem,lon_deg,lat_deg,aG,aGep,aGem,MG,MGep,MGem,mG,mGep,mGem,aBP,aBPep,aBPem,MBP,MBPep,MBPem,mBP,mBPep,mBPem,aRP,aRPep,aRPem,MRP,MRPep,MRPem,mRP,mRPep,mRPem,aB,aBep,aBem,MB,MBep,MBem,mB,mBep,mBem,aV,aVep,aVem,MV,MVep,MVem,mV,mVep,mVem,aJ,aJep,aJem,MJ,MJep,MJem,mJ,mJep,mJem,aH,aHep,aHem,MH,MHep,MHem,mH,mHep,mHem,aK,aKep,aKem,MK,MKep,MKem,mK,mKep,mKem
,deg,deg,mas,mas,mag,,mag,,mag,,mag,,mag,,mag,,mag,,mag,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
str21,float64,float64,float64,float32,float32,float64,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1022096392749670016,145.93959651092428,55.952388737534136,19.266700572347492,0.01887033,7.7452164,0.007181938969436053,8.088248,0.024051615865364074,7.2756886,0.02388009262163658,8.54,-99.000,7.75,-99.000,6.573,0.029,6.223,0.034,6.184,0.017,51.826133728027344,0.040660858154296875,0.044971466064453125,158.92257837077454,45.90248575416284,0.0,0.0,0.0,4.172472238540649,0.009067233883681247,0.008884843670668552,7.745216369628906,0.007181938969435642,0.007181938969435642,0.0,0.0,0.0,4.5155041217803955,0.02593691077960969,0.025754520566596995,8.088248252868652,0.024051615865364084,0.024051615865364084,0.0,0.0,0.0,3.70294451713562,0.02576538753588231,0.025582997322869616,7.275688648223877,0.023880092621636706,0.023880092621636706,0.0,0.0,0.0,4.9672558307647705,0.02188529491424518,0.021702904701232484,8.539999961853027,0.019999999999999574,0.019999999999999574,0.0,0.0,0.0,4.177255868911743,0.02188529491424518,0.021702904701232484,7.75,0.019999999999999574,0.019999999999999574,0.0,0.0,0.0,3.000255823135376,0.03088510036468506,0.030702710151672363,6.572999954223633,0.028999805450439453,0.028999805450439453,0.0,0.0,0.0,2.6502559185028076,0.03588521480560303,0.03570282459259033,6.2230000495910645,0.03399991989135742,0.03399991989135742,0.0,0.0,0.0,2.6112558841705322,0.018885493278503418,0.018703103065490723,6.184000015258789,0.017000198364257812,0.017000198364257812
3107142829863485952,101.76822041702867,-1.2075151625364267,0.432227248321794,0.030997224,12.621653,0.0006344411880043353,12.802992,0.0020885144822050884,12.291391,0.0017593474516210648,13.01,0.020,12.72,0.020,11.903,0.023,11.838,0.023,11.8,0.024,2193.359375,145.941162109375,138.426025390625,213.5092718420073,-1.5216790901099373,1.0972506348348021,0.2147918691552715,0.01373223426890191,-0.181146794960533,0.15592651470556262,0.3553076827950612,11.524401968314612,0.014366675456907174,0.2154263103432772,1.3107321146267983,0.25658185279147117,0.01640398272773691,-0.2132890108365153,0.16005233645859995,0.3985517397254572,11.492259752438631,0.018492497209942726,0.25867036727367676,0.7576583159506094,0.14831510750373622,0.009482192272307954,-0.17181570654509315,0.15280137897258683,0.28995582740713743,11.533733056730055,0.01124153972392783,0.15007445495535698,1.5085403078773538,0.2953037183467466,0.018879577968958783,-0.20408884227066437,0.180439874981289,0.45518554856220206,11.501459921004482,0.03888003573262999,0.31530417611041806,1.1924453025900386,0.23342666413430613,0.014923607905210634,-0.17799379883637556,0.1764839049175393,0.39330849434976045,11.527554964438771,0.034924065

### Querying DR2 catalog

If DR2 catalog is queried, then __Gaia DR2 IDs__ must be passed.

In [6]:
result_table_dr2 = query_gaia(targets,gaiaDR=2)

Using Combined19 map from mwdust
Querying Gaia DR2 catalog...
Calculating distances, absolute magnitudes...


  0%|          | 0/2 [00:00<?, ?it/s]

The result is an Astropy Table with distance probabilistically estimated from parallaxes.

In [7]:
result_table_dr2

Source,ra,dec,parallax,sig_parallax,Gmag,sig_Gmag,BPmag,sig_BPmag,RPmag,sig_RPmag,Bmag,sig_Bmag,Vmag,sig_Vmag,Jmag,sig_Jmag,Hmag,sig_Hmag,Kmag,sig_Kmag,dist,distep,distem,lon_deg,lat_deg,aG,aGep,aGem,MG,MGep,MGem,mG,mGep,mGem,aBP,aBPep,aBPem,MBP,MBPep,MBPem,mBP,mBPep,mBPem,aRP,aRPep,aRPem,MRP,MRPep,MRPem,mRP,mRPep,mRPem,aB,aBep,aBem,MB,MBep,MBem,mB,mBep,mBem,aV,aVep,aVem,MV,MVep,MVem,mV,mVep,mVem,aJ,aJep,aJem,MJ,MJep,MJem,mJ,mJep,mJem,aH,aHep,aHem,MH,MHep,MHem,mH,mHep,mHem,aK,aKep,aKem,MK,MKep,MKem,mK,mKep,mKem
,deg,deg,mas,mas,mag,,mag,,mag,,mag,,mag,,mag,,mag,,mag,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
str21,float64,float64,float64,float64,float32,float64,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1022096392749670016,145.93959226288555,55.95239269960827,19.277532306604552,0.03343194611367065,7.7937016,0.012570032498626791,8.1552515,0.038804118205489575,7.367436,0.03560460850334358,8.54,-99.000,7.75,-99.000,6.573,0.029,6.223,0.034,6.184,0.017,51.87414874148742,0.0891440914409074,0.08908409084090607,158.9225742720827,45.902482118266626,0.0,0.0,0.0,4.2188973476900475,0.013046969776703499,0.013010926166682424,7.793674517138414,0.012492582713060862,0.012492948098652867,0.0,0.0,0.0,4.580228054832856,0.03885830393406131,0.03852396343961306,8.155167746899021,0.038565028081066544,0.03856615603882929,0.0,0.0,0.0,3.79242611742484,0.035703094077422115,0.035385582689146045,7.367359081995247,0.03538523203840338,0.03538626699284464,0.0,0.0,0.0,4.965118069467221,0.020282138388814985,0.02018711115041416,8.539956793191774,0.019876770747291772,0.019877352107114277,0.0,0.0,0.0,4.175118107614193,0.020282138388814985,0.02018711115041416,7.749956831338746,0.01987677074729266,0.01987735210711339,0.0,0.0,0.0,2.998025292249327,0.0291551561762855,0.028906115384708997,6.572937359666616,0.028821316754251036,0.02882215972596658,0.0,0.0,0.0,2.648006946167431,0.03411431133917109,0.03382855899612647,6.222926662862945,0.033790512106756765,0.03379150041850654,0.0,0.0,0.0,2.609154383087451,0.017350237770220378,0.017266900610069413,6.18396332189473,0.016895256053377494,0.016895750209253713
3107142829863485952,101.76822078135328,-1.207515192087555,0.2784408581785764,0.07150647466161306,12.631575,0.0009382113738554969,12.801229,0.0038396854144956893,12.298964,0.002020975902707495,13.01,0.020,12.72,0.020,11.903,0.023,11.838,0.023,11.8,0.024,3832.632796327963,1205.1134011340114,781.2820460204603,213.50927203436055,-1.521678779347881,1.7696987325866222,0.4122728676707361,0.2407382729227452,-2.0556418277891266,0.7348429400105714,1.005430029894729,10.861819577586186,0.2396506499506046,0.4117372038317537,2.1140119571358085,0.49248482569953156,0.2875763982466415,-2.2304442769199397,0.7812866080966312,1.0848303808647772,10.687120093832506,0.28631905727537976,0.49173506812071466,1.221987865765398,0.2846769522977459,0.1662312589821069,-1.8407000304353147,0.660724716880748,0.8775676772207697,11.07694807645693,0.16551958658712707,0.28428683795836207,2.4330465493950886,0.5668078185428684,0.3309757833108171,-2.341223028975283,0.8260060572224985,1.1580442280453713,10.57684229168457,0.3304529423714726,0.5664240439997599,1.923233282968368,0.44804061062430733,0.2616241117450673,-2.1210607288462935,0.7566343652693712,1.0398941209468324,10.796809821669349,0.261295059028967

### Selecting the dust map

There are several dust maps implemented in `mwdust`. By default the _Combined19_ map is used, however any other available map can also be selected. E.g. changing the map to Green19 can be done by passing the name of the map to `dustmodel`.

__Note:__ the desired dust map must be downloaded beforehand.

In [8]:
result_table = query_gaia(targets,dustmodel='Green19')

Using Green19 map from mwdust
Querying Gaia DR3 and BJ catalogs...
Calculating distances, absolute magnitudes...


  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
result_table

Source,ra,dec,parallax,sig_parallax,Gmag,sig_Gmag,BPmag,sig_BPmag,RPmag,sig_RPmag,Bmag,sig_Bmag,Vmag,sig_Vmag,Jmag,sig_Jmag,Hmag,sig_Hmag,Kmag,sig_Kmag,dist,distep,distem,lon_deg,lat_deg,aG,aGep,aGem,MG,MGep,MGem,mG,mGep,mGem,aBP,aBPep,aBPem,MBP,MBPep,MBPem,mBP,mBPep,mBPem,aRP,aRPep,aRPem,MRP,MRPep,MRPem,mRP,mRPep,mRPem,aB,aBep,aBem,MB,MBep,MBem,mB,mBep,mBem,aV,aVep,aVem,MV,MVep,MVem,mV,mVep,mVem,aJ,aJep,aJem,MJ,MJep,MJem,mJ,mJep,mJem,aH,aHep,aHem,MH,MHep,MHem,mH,mHep,mHem,aK,aKep,aKem,MK,MKep,MKem,mK,mKep,mKem
,deg,deg,mas,mas,mag,,mag,,mag,,mag,,mag,,mag,,mag,,mag,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
str21,float64,float64,float64,float32,float32,float64,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1022096392749670016,145.93959651092428,55.952388737534136,19.266700572347492,0.01887033,7.7452164,0.007181938969436053,8.088248,0.024051615865364074,7.2756886,0.02388009262163658,8.54,-99.000,7.75,-99.000,6.573,0.029,6.223,0.034,6.184,0.017,51.831443786621094,0.0460662841796875,0.050594329833984375,158.92257837077454,45.90248575416284,0.0,0.0,0.0,4.172249913215637,0.00930267223054404,0.009111341320815036,7.745216369628906,0.007181938969435642,0.007181938969435642,0.0,0.0,0.0,4.515281796455383,0.026172349126472483,0.02598101821674348,8.088248252868652,0.024051615865364084,0.024051615865364084,0.0,0.0,0.0,3.702722191810608,0.026000825882745104,0.0258094949730161,7.275688648223877,0.023880092621636706,0.023880092621636706,0.0,0.0,0.0,4.967033505439758,0.022120733261107972,0.021929402351378968,8.539999961853027,0.019999999999999574,0.019999999999999574,0.0,0.0,0.0,4.177033543586731,0.022120733261107972,0.021929402351378968,7.75,0.019999999999999574,0.019999999999999574,0.0,0.0,0.0,3.0000334978103638,0.03112053871154785,0.030929207801818848,6.572999954223633,0.028999805450439453,0.028999805450439453,0.0,0.0,0.0,2.6500335931777954,0.03612065315246582,0.035929322242736816,6.2230000495910645,0.03399991989135742,0.03399991989135742,0.0,0.0,0.0,2.61103355884552,0.01912093162536621,0.018929600715637207,6.184000015258789,0.017000198364257812,0.017000198364257812
3107142829863485952,101.76822041702867,-1.2075151625364267,0.432227248321794,0.030997224,12.621653,0.0006344411880043353,12.802992,0.0020885144822050884,12.291391,0.0017593474516210648,13.01,0.020,12.72,0.020,11.903,0.023,11.838,0.023,11.8,0.024,2190.859619140625,197.47900390625,151.5408935546875,213.5092718420073,-1.5216790901099373,1.0972506348348021,0.2813463101744822,0.01746006095976571,-0.17867081801656326,0.17374130314264136,0.46938729127948164,11.524401968314612,0.018094502147770086,0.2819807513624877,1.3107321146267983,0.3360851498919084,0.020857096725901858,-0.21081303389254558,0.1785924122029794,0.5255802042911046,11.492259752438631,0.022945611208106342,0.33817366437411245,0.7576583159506094,0.19427135861061418,0.012056279543791626,-0.16933972960112342,0.1694624279902861,0.383437245979227,11.533733056730055,0.013815626995413055,0.1960307060622366,1.5085403078773538,0.3868051982805827,0.024004730459570967,-0.20161286532669465,0.19965198921811478,0.5942121959612479,11.501459921004482,0.04400518822324351,0.4068056560442539,1.1924453025900386,0.305755198782921,0.01897485133607879,-0.17551782189240583,0.19462211009462393,0.5131621964635862,11.527554964438771,0.0389753090

### Setting the parallax offset in Gaia DR2

A parallax offset can be given if the Gaia DR2 catalog is used. There are four predefined options, which can be selected:

- __Stassun__ : +0.08 mas (Stassun et al. 2018)
- __Riess__ : +0.046  mas (Riess et al. 2018)
- __BJ__ : +0.029  mas (Bailer-Jones et al. 2018)
- __Zinn__ : +0.0528 mas (Zinn et al 2019)
        
or simply an offset in _mas_ can be passed.

In [10]:
result_table = query_gaia(targets,gaiaDR=2,plx_offset='Stassun')

Using Combined19 map from mwdust
Querying Gaia DR2 catalog...
Calculating distances, absolute magnitudes...


  0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
result_table

Source,ra,dec,parallax,sig_parallax,Gmag,sig_Gmag,BPmag,sig_BPmag,RPmag,sig_RPmag,Bmag,sig_Bmag,Vmag,sig_Vmag,Jmag,sig_Jmag,Hmag,sig_Hmag,Kmag,sig_Kmag,dist,distep,distem,lon_deg,lat_deg,aG,aGep,aGem,MG,MGep,MGem,mG,mGep,mGem,aBP,aBPep,aBPem,MBP,MBPep,MBPem,mBP,mBPep,mBPem,aRP,aRPep,aRPem,MRP,MRPep,MRPem,mRP,mRPep,mRPem,aB,aBep,aBem,MB,MBep,MBem,mB,mBep,mBem,aV,aVep,aVem,MV,MVep,MVem,mV,mVep,mVem,aJ,aJep,aJem,MJ,MJep,MJem,mJ,mJep,mJem,aH,aHep,aHem,MH,MHep,MHem,mH,mHep,mHem,aK,aKep,aKem,MK,MKep,MKem,mK,mKep,mKem
,deg,deg,mas,mas,mag,,mag,,mag,,mag,,mag,,mag,,mag,,mag,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
str21,float64,float64,float64,float64,float32,float64,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1022096392749670016,145.93959226288555,55.95239269960827,19.277532306604552,0.03343194611367065,7.7937016,0.012570032498626791,8.1552515,0.038804118205489575,7.367436,0.03560460850334358,8.54,-99.000,7.75,-99.000,6.573,0.029,6.223,0.034,6.184,0.017,51.65976659766598,0.08840408404083888,0.08836088360883565,158.9225742720827,45.902482118266626,0.0,0.0,0.0,4.227894182210269,0.013030806596175282,0.013012296249788946,7.793674517138414,0.012492582713060862,0.012492948098652867,0.0,0.0,0.0,4.589225743008126,0.03885348450892767,0.038518885345538934,8.155167746899021,0.038565028081066544,0.03856615603882929,0.0,0.0,0.0,3.8014235319409115,0.0356994466157885,0.035379920733777936,7.367359081995247,0.03538523203840338,0.03538626699284464,0.0,0.0,0.0,4.974106774584016,0.020284509688115904,0.02017683465475084,8.539956793191774,0.019876770747291772,0.019877352107114277,0.0,0.0,0.0,4.184106812730989,0.020284509688115904,0.020176834654749953,7.749956831338746,0.01987677074729266,0.01987735210711339,0.0,0.0,0.0,3.0070175491396705,0.029152333025087795,0.028905708755822435,6.572937359666616,0.028821316754251036,0.02882215972596658,0.0,0.0,0.0,2.657003097080757,0.03411146917043251,0.033822337191342644,6.222926662862945,0.033790512106756765,0.03379150041850654,0.0,0.0,0.0,2.618143657011534,0.017349068044365534,0.017264308333109213,6.18396332189473,0.016895256053377494,0.016895750209253713
3107142829863485952,101.76822078135328,-1.207515192087555,0.2784408581785764,0.07150647466161306,12.631575,0.0009382113738554969,12.801229,0.0038396854144956893,12.298964,0.002020975902707495,13.01,0.020,12.72,0.020,11.903,0.023,11.838,0.023,11.8,0.024,3002.1165711657113,774.5859362593628,522.2633730337302,213.50927203436055,-1.521678779347881,1.5100084833778324,0.25507143021589407,0.12459642264064508,-1.2656104946923428,0.5395534174390507,0.7529658084015112,11.121501819021775,0.12454852596685662,0.2545465746165796,1.8037962792522915,0.3046982197992494,0.1488379476283419,-1.3899511901198425,0.5640108325645211,0.8018706177346147,10.997197477331621,0.14910453299335913,0.30397998389938685,1.0426701505252982,0.17612839230080168,0.08603459661301027,-1.1310405663542102,0.501124140111731,0.6737948142508945,11.256183914643884,0.08614492686314534,0.1756999121809386,2.0760148958630262,0.3506815322340038,0.17129971931985621,-1.4539404226702475,0.5876219169390448,0.846843200061425,10.933519197228057,0.1787145637530454,0.3499675195094909,1.6410129697907134,0.2772007772241196,0.13540609062370623,-1.3088457606127673,0.5518162109318188,0.7734214697422517,11.078628340896056,0.